In [1]:
#全局常量
repo_id = 'lansinuote/diffusion.3.dream_booth'
checkpoint = 'runwayml/stable-diffusion-v1-5'

In [2]:
from datasets import load_dataset

#直接使用我处理好的数据集
dataset = load_dataset(path=repo_id, split='train')

dataset, dataset[0]

Using custom data configuration lansinuote--diffusion.3.dream_booth-4344a7d7501be7f1
Found cached dataset parquet (/root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--diffusion.3.dream_booth-4344a7d7501be7f1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


(Dataset({
     features: ['image', 'text'],
     num_rows: 5
 }),
 {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2469x2558>,
  'text': 'a photo of little dog'})

In [3]:
import torchvision
from transformers import AutoTokenizer

#数据增强
compose = torchvision.transforms.Compose([
    torchvision.transforms.Resize(
        512, interpolation=torchvision.transforms.InterpolationMode.BILINEAR),
    torchvision.transforms.RandomCrop(512),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5], [0.5]),
])

#文字编码
tokenizer = AutoTokenizer.from_pretrained(checkpoint,
                                          subfolder='tokenizer',
                                          use_fast=False)


def f(data):
    #图像编码
    pixel_values = compose(data['image'][0]).unsqueeze(dim=0)

    #文字编码
    #77 = tokenizer.model_max_length
    tokens = tokenizer(data['text'][0],
                       truncation=True,
                       padding='max_length',
                       max_length=77,
                       return_tensors='pt')

    return {
        'pixel_values': pixel_values,
        'input_ids': tokens.input_ids,
        'attention_mask': tokens.attention_mask
    }


dataset = dataset.with_transform(f)

for k, v in dataset[0].items():
    print(k, v.shape, v.dtype)

dataset

pixel_values torch.Size([3, 512, 512]) torch.float32
input_ids torch.Size([77]) torch.int64
attention_mask torch.Size([77]) torch.int64


Dataset({
    features: ['image', 'text'],
    num_rows: 5
})

In [4]:
import torch

loader = torch.utils.data.DataLoader(dataset,
                                     batch_size=1,
                                     shuffle=True,
                                     collate_fn=None)

for k, v in next(iter(loader)).items():
    print(k, v.shape, v.dtype)

len(loader)

pixel_values torch.Size([1, 3, 512, 512]) torch.float32
input_ids torch.Size([1, 77]) torch.int64
attention_mask torch.Size([1, 77]) torch.int64


5

In [5]:
from transformers.models.clip.modeling_clip import CLIPTextModel
from diffusers import AutoencoderKL, UNet2DConditionModel

encoder = CLIPTextModel.from_pretrained(checkpoint, subfolder='text_encoder')
vae = AutoencoderKL.from_pretrained(checkpoint, subfolder='vae')
unet = UNet2DConditionModel.from_pretrained(checkpoint, subfolder='unet')

vae.requires_grad_(False)
encoder.requires_grad_(False)
unet.requires_grad_(False)


def print_model_size(name, model):
    print(name, sum(i.numel() for i in model.parameters()) / 10000)


print_model_size('encoder', encoder)
print_model_size('vae', vae)
print_model_size('unet', unet)

encoder 12306.048
vae 8365.3863
unet 85952.0964


In [6]:
#这个类不想测了,总之就是在做注意力的调整
from diffusers.models.cross_attention import LoRACrossAttnProcessor


def set_processors():
    processors = {}

    #遍历unet的所有层,找出所有有set_processor属性的层,每一个都组装成lora层
    for name in unet.attn_processors.keys():
        #768 = unet.config.cross_attention_dim
        cross_attention_dim = 768
        if name.endswith('attn1.processor'):
            cross_attention_dim = None

        #1280 = unet.config.block_out_channels[-1]
        hidden_size = 1280

        if name.startswith('up_blocks'):
            #取层名字中的第一个数字
            #p_blocks.1.attentions.0.transformer_blocks.0.attn1.processor -> 1
            block_id = int(name[10])
            hidden_size = list(reversed(
                unet.config.block_out_channels))[block_id]

        if name.startswith('down_blocks'):
            #取层名字中的第一个数字
            #down_blocks.2.attentions.0.transformer_blocks.0.attn1.processor -> 2
            block_id = int(name[12])
            hidden_size = unet.config.block_out_channels[block_id]

        processors[name] = LoRACrossAttnProcessor(hidden_size,
                                                  cross_attention_dim)

        print(name, hidden_size, cross_attention_dim)

    #把上面组装好的字典,设置到unet的层当中
    unet.set_attn_processor(processors)


set_processors()

down_blocks.0.attentions.0.transformer_blocks.0.attn1.processor 320 None
down_blocks.0.attentions.0.transformer_blocks.0.attn2.processor 320 768
down_blocks.0.attentions.1.transformer_blocks.0.attn1.processor 320 None
down_blocks.0.attentions.1.transformer_blocks.0.attn2.processor 320 768
down_blocks.1.attentions.0.transformer_blocks.0.attn1.processor 640 None
down_blocks.1.attentions.0.transformer_blocks.0.attn2.processor 640 768
down_blocks.1.attentions.1.transformer_blocks.0.attn1.processor 640 None
down_blocks.1.attentions.1.transformer_blocks.0.attn2.processor 640 768
down_blocks.2.attentions.0.transformer_blocks.0.attn1.processor 1280 None
down_blocks.2.attentions.0.transformer_blocks.0.attn2.processor 1280 768
down_blocks.2.attentions.1.transformer_blocks.0.attn1.processor 1280 None
down_blocks.2.attentions.1.transformer_blocks.0.attn2.processor 1280 768
up_blocks.1.attentions.0.transformer_blocks.0.attn1.processor 1280 None
up_blocks.1.attentions.0.transformer_blocks.0.attn2.pr

/root/anaconda3/envs/pt39/lib/python3.9/site-packages/diffusers/models/cross_attention.py:30: FutureWarning: Importing from cross_attention is deprecated. Please import from diffusers.models.attention_processor instead.
  deprecate(
/root/anaconda3/envs/pt39/lib/python3.9/site-packages/diffusers/models/cross_attention.py:58: FutureWarning: LoRACrossAttnProcessor is deprecated and will be removed in `0.18.0`. Please use `from diffusers.models.attention_processor import LoRAAttnProcessor instead.
  deprecate("cross_attention", "0.18.0", deprecation_message, standard_warn=False)


In [7]:
lora_layers = torch.nn.ModuleList(unet.attn_processors.values())

len(lora_layers)

32

In [8]:
from diffusers import DDPMScheduler

scheduler = DDPMScheduler.from_pretrained(checkpoint, subfolder='scheduler')

optimizer = torch.optim.AdamW(lora_layers.parameters(),
                              lr=1e-4,
                              betas=(0.9, 0.999),
                              weight_decay=0.01,
                              eps=1e-8)

criterion = torch.nn.MSELoss()

scheduler, optimizer, criterion

(DDPMScheduler {
   "_class_name": "DDPMScheduler",
   "_diffusers_version": "0.15.1",
   "beta_end": 0.012,
   "beta_schedule": "scaled_linear",
   "beta_start": 0.00085,
   "clip_sample": false,
   "clip_sample_range": 1.0,
   "dynamic_thresholding_ratio": 0.995,
   "num_train_timesteps": 1000,
   "prediction_type": "epsilon",
   "sample_max_value": 1.0,
   "set_alpha_to_one": false,
   "skip_prk_steps": true,
   "steps_offset": 1,
   "thresholding": false,
   "trained_betas": null,
   "variance_type": "fixed_small"
 },
 AdamW (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     eps: 1e-08
     foreach: None
     lr: 0.0001
     maximize: False
     weight_decay: 0.01
 ),
 MSELoss())

In [9]:
def get_loss(data):
    device = data['input_ids'].device

    #编码文字
    #[1, 77] -> [1, 77, 768]
    out_encoder = encoder(input_ids=data['input_ids'],
                          attention_mask=data['attention_mask'])[0]

    #vae计算特征图
    #[1, 3, 512, 512] -> [1, 4, 64, 64]
    out_vae = vae.encode(data['pixel_values']).latent_dist.sample().detach()

    #0.18215 = vae.config.scaling_factor
    out_vae = out_vae * 0.18215

    #随机噪声
    #[1, 4, 64, 64]
    noise = torch.randn_like(out_vae)

    #随机噪声步
    #1000 = scheduler.config.num_train_timesteps
    #1 = b
    noise_step = torch.randint(0, 1000, (1, ), device=device).long()

    #添加噪声
    #[1, 4, 64, 64]
    out_vae_noise = scheduler.add_noise(out_vae, noise, noise_step)

    #unet从噪声图中把噪声计算出来
    #[1, 4, 64, 64],[1, 77, 768] -> [1, 4, 64, 64]
    out_unet = unet(out_vae_noise, noise_step, out_encoder).sample

    return criterion(out_unet, noise)


get_loss({
    'pixel_values': torch.randn(1, 3, 512, 512),
    'input_ids': torch.ones(1, 77).long(),
    'attention_mask': torch.ones(1, 77).long()
})

tensor(0.2743, grad_fn=<MseLossBackward0>)

In [10]:
from diffusers import DiffusionPipeline


def train():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    unet.to(device)
    vae.to(device)
    encoder.to(device)
    unet.train()

    loss_sum = 0
    for epoch in range(200):
        for i, data in enumerate(loader):
            for k in data.keys():
                data[k] = data[k].to(device)
            loss = get_loss(data)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(lora_layers.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()

            loss_sum += loss.item()

        if epoch % 10 == 0:
            print(epoch, loss_sum)
            loss_sum = 0

    unet.save_attn_procs('./save')


train()

0 0.47430265601724386
10 6.008332805009559
20 5.410161764128134
30 4.312515129800886
40 5.0653923387872055
50 4.800978829618543
60 4.101198305841535
70 3.55872066388838
80 5.382800737861544
90 3.1098747176583856
100 5.2366352655226365
110 3.251381019828841
120 6.391375361359678
130 3.4201086544198915
140 3.95328053063713
150 4.018299808958545
160 4.696397997322492
170 3.5135789506603032
180 4.713769806083292
190 3.8237785125384107
